In [1]:
#!pip install openai scikit-learn pandas numpy --quiet

In [2]:
import openai
import os
from google.colab import userdata


openai.api_key =userdata.get('OPEN_API_KEY')

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

def generate_metrics():
    df = pd.DataFrame({
        "cpu": np.random.normal(50, 10, 100),
        "memory": np.random.normal(60, 15, 100),
        "disk_io": np.random.normal(100, 25, 100),
    })
    df.iloc[95:100] += np.random.uniform(100, 200)
    return df

def detect_anomalies(df):
    model = IsolationForest(contamination=0.05, random_state=42)
    model.fit(df)
    df["anomaly"] = model.predict(df)
    return df[df["anomaly"] == -1]


In [4]:
def build_messages(anomalies_df):
    return [
        {"role": "system", "content": "You are an expert in DevOps and anomaly detection."},
        {"role": "user", "content": f"""Detected anomalies in system metrics:

{anomalies_df.to_string(index=False)}

Please:
1. Identify likely causes
2. Describe potential system impact
3. Suggest recommended actions
"""}
    ]


In [5]:
import time

def evaluate_response(text, start_time):
    duration = round(time.time() - start_time, 2)
    words = len(text.split())
    keywords = ["cpu", "memory", "disk", "restart", "fix", "usage", "overload"]
    relevance_hits = sum(k in text.lower() for k in keywords)
    relevance_score = round(relevance_hits / len(keywords), 2)
    return {
        "latency_seconds": duration,
        "word_count": words,
        "relevance_score": relevance_score
    }


In [6]:
df = generate_metrics()
anomalies = detect_anomalies(df)

if anomalies.empty:
    print("✅ No anomalies detected.")
else:
    print("🚨 Anomalies found. Sending to GPT...\n")
    messages = build_messages(anomalies)

    start = time.time()
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",  # Change to gpt-4 or gpt-4o if you have access
        messages=messages
    )
    reply = response.choices[0].message.content.strip()

    print("🧠 ChatGPT Response:\n")
    print(reply)

    metrics = evaluate_response(reply, start)
    print("\n📊 Evaluation Metrics:")
    for k, v in metrics.items():
        print(f" - {k}: {v}")


🚨 Anomalies found. Sending to GPT...

🧠 ChatGPT Response:

Based on the provided system metrics and detected anomalies, here are some insights and recommendations:

1. **Identified Anomalies**:
    - Anomalies have been detected in the CPU, memory, and disk IO metrics.
    - The anomalies are indicated by the value of -1 in the "anomaly" column.

2. **Likely Causes**:
    - Sudden spikes or drops in the metrics might be caused by processes consuming excessive resources.
    - Incorrect resource allocation or inefficient usage by applications could lead to anomalies.
    - Hardware failures or issues in the underlying infrastructure could also contribute to anomalies.

3. **Potential System Impact**:
    - High CPU usage can lead to performance degradation, slow response times, or even system crashes.
    - Memory anomalies may result in memory leaks, leading to out-of-memory errors and system instability.
    - Disk IO anomalies might slow down data read/write operations, impacting ove